In [1]:
import os
import rasterio
import numpy as np
from osgeo import gdal, gdalconst
import pandas as pd
from shutil import copy2
from tqdm import tqdm

In [9]:
S2_dir = r'G:\Thesis_Code\Data_Analysis\2_PreProcessing\S2Imagery\originalS2'
S2_B02_dir = os.path.join(S2_dir, '20190121\S2A_MSIL2A_20190120T103341_N0211_R108_T31TFJ_20190120T131644.SAFE\GRANULE\L2A_T31TFJ_A018695_20190120T103339\IMG_DATA\R20m')
original_B08_dir = r'G:\Thesis_Code\Data_Analysis\2_PreProcessing\S2Imagery\Step0_ResampleB08\inputFile'
ref_B02file_dir = r'G:\Thesis_Code\Data_Analysis\2_PreProcessing\S2Imagery\Step0_ResampleB08\refFile'
resampled_B08_dir = r'G:\Thesis_Code\Data_Analysis\2_PreProcessing\S2Imagery\Step0_ResampleB08\outputFile'
preparedS2_forGF_dir = r'G:\Thesis_Code\Data_Analysis\2_PreProcessing\gapFilling\prepareS2\Step1_ApplySCLMask\inputFiles'
cloudMaskedS2 = r'G:\Thesis_Code\Data_Analysis\2_PreProcessing\gapFilling\prepareS2\Step1_ApplySCLMask\cloudMaskedS2'
resampling_cloudMaskedS2_dir = r'G:\Thesis_Code\Data_Analysis\2_PreProcessing\gapFilling\prepareS2\Step2_AssignCRS_Resample_Clip'
bathymetryMask_dir = r'G:\Thesis_Code\Data_Analysis\2_PreProcessing\bathymetry\binaryMask'
landMaskedS2 = r'G:\Thesis_Code\Data_Analysis\2_PreProcessing\gapFilling\prepareS2\Step3_ApplyElevMask'

In [3]:
#########################################################################
# Resampling B08 (so it shares the same spatial res. as other S2 bands) #
#########################################################################

# Copy original B08 (10m) files to another directory
for fileName1 in os.listdir(S2_dir):
    subdir1 = os.path.join(S2_dir,fileName1)
    for fileName2 in os.listdir(subdir1):
        date = fileName2[11:19]
        subdir2 = os.path.join(subdir1,fileName2,'GRANULE')
        for fileName3 in os.listdir(subdir2):
            subdir3 = os.path.join(subdir2,fileName3,'IMG_DATA','R10m')
            for fileName4 in os.listdir(subdir3):
                if 'B08' in fileName4:
                    src1 = os.path.join(subdir3,fileName4)
                    copy2(src1, original_B08_dir)

# Copy one original B02 (20m) file to another directory # to be used as reference
B02_file = 'T31TFJ_20190120T103341_B02_20m.jp2'
src2 = os.path.join(S2_B02_dir, B02_file)
copy2(src2, ref_B02file_dir)

# Resample B08 from 10m to 20m using a reference B02 image
reference = gdal.Open(os.path.join(ref_B02file_dir ,B02_file), gdalconst.GA_ReadOnly)
referenceProj = reference.GetProjection()
referenceTrans = reference.GetGeoTransform()
bandreference = reference.GetRasterBand(1)    
x = reference.RasterXSize 
y = reference.RasterYSize
    
for fileName in os.listdir(original_B08_dir):
    inputfile = gdal.Open(os.path.join(original_B08_dir, fileName), gdalconst.GA_ReadOnly)
    inputProj = inputfile.GetProjection()
    inputTrans = inputfile.GetGeoTransform()
    
    outputfile = os.path.join(resampled_B08_dir,fileName[:-8]+'_20m'+'.tiff')
    driver= gdal.GetDriverByName('GTiff')
    output = driver.Create(outputfile,x,y,1,bandreference.DataType)
    output.SetGeoTransform(referenceTrans)
    output.SetProjection(referenceProj)
    # options: nearest neighbor, bilinear, cubic
    gdal.ReprojectImage(inputfile,output,inputProj,referenceProj,gdalconst.GRA_Bilinear) 

inputfile.FlushCache()
inputfile = None
del output

# Copy resampled B08 files to another directory
for fileName in os.listdir(resampled_B08_dir):    
    src = os.path.join(resampled_B08_dir,fileName)
    dst = os.path.join(preparedS2_forGF_dir,fileName[7:15])
    os.makedirs(dst, exist_ok=True) # Creates the folder if it does not exist
    copy2(src, dst)

In [6]:
################################################
# Assign -99 to clouded pixels using SCL files #
################################################
# Copy original band (20m) files to another directory
for fileName1 in os.listdir(S2_dir):
    subdir1 = os.path.join(S2_dir,fileName1)
    for fileName2 in os.listdir(subdir1):
        date = fileName2[11:19]
        subdir2 = os.path.join(subdir1,fileName2,'GRANULE')
        for fileName3 in os.listdir(subdir2):
            subdir3 = os.path.join(subdir2,fileName3,'IMG_DATA','R20m')
            for fileName4 in os.listdir(subdir3):
                if 'B02' in fileName4 or 'B03' in fileName4 or 'B04' in fileName4 or 'B05' in fileName4 or 'B06' in fileName4 or 'B07' in fileName4 or 'B8A' in fileName4 or 'SCL' in fileName4:
                    src = os.path.join(subdir3,fileName4)
                    dst = os.path.join(preparedS2_forGF_dir,fileName4[7:15])
                    os.makedirs(dst, exist_ok=True) # Creates the folder if it does not exist
                    copy2(src, dst)

# Assign -99 to clouded pixels
for i in tqdm(range(len(os.listdir(preparedS2_forGF_dir)))):
    fileName1 = os.listdir(preparedS2_forGF_dir)[i]
    subdir1 = os.path.join(preparedS2_forGF_dir,fileName1)
    # Transform the SCL layer into a binary mask
    scl = rasterio.open(os.path.join(subdir1,os.listdir(subdir1)[-1])).read() # Read the corresponding SCL file
    vals = np.unique(scl).tolist()
    other_values = vals.copy()
    shadow_cloud_values = [3.0, 8.0, 9.0, 10.0]  # List of shadow and cloud values                              
    for item in shadow_cloud_values:
        if item in other_values:
            other_values.remove(item)    # List of values other than shadow and clouds                                          
    for item in other_values:
        scl[scl==item] = 1
    for item in shadow_cloud_values:
        scl[scl==item] = 0
    # Apply mask on the 8 bands
    for fileName2 in os.listdir(subdir1)[:8]:
        img = rasterio.open(os.path.join(subdir1, fileName2))
        img_arr = img.read() 
        cloudMask = np.array(scl)
        new_img = np.multiply(cloudMask, img_arr).astype(np.float32)
        new_img[new_img==0]=-99

        subdir2 = os.path.join(cloudMaskedS2,fileName1)
        os.makedirs(subdir2, exist_ok=True) # Creates the folder if it does not exist
        outputdir = os.path.join(subdir2, fileName2[7:30]+'.tiff')
        with rasterio.open(outputdir,'w',driver='Gtiff', width=img.width, height=img.height, 
                      count=1,crs=img.crs,transform=img.transform, dtype=np.float32, nodata=9.96921e+36) as newImg:
            newImg.write(new_img)    
            newImg.close()

100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [18:57<00:00, 189.64s/it]


In [7]:
##################################################
# Reproject+Resample+Clip cloud masked S2 images #
##################################################

# Read turbidity file to be used for reprojection
tur_dir = r'G:\Thesis_Code\Data_Analysis\2_PreProcessing\copernicusTurLayers'
tur_file = 'c_gls_LWQ100-turbidity-blended-mean_201902110000_CUSTOM_MSI_V1.3.tiff'
reference = gdal.Open(os.path.join(tur_dir, tur_file), gdalconst.GA_ReadOnly)
referenceProj = reference.GetProjection()
referenceTrans = reference.GetGeoTransform()
bandreference = reference.GetRasterBand(1)    
x = reference.RasterXSize 
y = reference.RasterYSize

# Resample
for fileName1 in os.listdir(cloudMaskedS2):
    subdir1 = os.path.join(cloudMaskedS2, fileName1)
    for fileName2 in os.listdir(subdir1):        
        inputfile = gdal.Open(os.path.join(subdir1, fileName2), gdalconst.GA_ReadOnly)    
        inputProj = inputfile.GetProjection()
        inputTrans = inputfile.GetGeoTransform()

        subdir2 = os.path.join(resampling_cloudMaskedS2_dir, fileName1)
        os.makedirs(subdir2, exist_ok=True) # Creates the folder if it does not exist
        
        outputfile = os.path.join(subdir2,'resampled_'+fileName2)
        driver= gdal.GetDriverByName('GTiff')
        output = driver.Create(outputfile,x,y,1,bandreference.DataType)
        output.SetGeoTransform(referenceTrans)
        output.SetProjection(referenceProj)
        # choose nearest neighbor to avoid averaging with -99
        gdal.ReprojectImage(inputfile,output,inputProj,referenceProj,gdalconst.GRA_NearestNeighbour) 

inputfile.FlushCache()
inputfile = None  
del output

In [10]:
###################
# Apply Land Mask #
###################

bathymetryMask_dir = r'G:\MScThesis\waterQualityMonitoring\Data\Bathymetry\France\binaryMask'
mask_img = rasterio.open(os.path.join(bathymetryMask_dir, 'binaryMask_v3.tiff'))
mask_arr = mask_img.read(1)

names = np.array([]).astype('str')
percent = np.array([]).astype('float')
for fileName1 in os.listdir(resampling_cloudMaskedS2_dir):
    subdir1 = os.path.join(resampling_cloudMaskedS2_dir,fileName1)    
    for fileName2 in os.listdir(subdir1):
        img = rasterio.open(os.path.join(subdir1, fileName2))
        arr = img.read(1)
        masked = np.multiply(mask_arr, arr)
        # calculate cloud%
        allNumPixels = np.sum(masked!=0) # nan values
        cloudPixels = np.sum(masked==-99)
        percentage = np.round(100*(cloudPixels/allNumPixels), 2)
        # Save file
        masked[masked==0]=9.96921e+36 # permanent change for nodata values
        
        subdir2 = os.path.join(landMaskedS2, fileName1)
        os.makedirs(subdir2, exist_ok=True) # Creates the folder if it does not exist
        
        outputdir = os.path.join(subdir2, 'masked_'+fileName2[10:])
        with rasterio.open(outputdir,'w',driver='Gtiff', width=img.width, height=img.height, 
                      count=1,crs=img.crs,transform=img.transform, dtype=np.float32, nodata=9.96921e+36) as newImg:
            newImg.write(masked, indexes=1)    
            newImg.close()
    names = np.append(names, fileName1)
    percent = np.append(percent, percentage)
# Save percentage file
df = pd.DataFrame({'Image': names,'Cloud Percentage': percent}) 
outputdir = os.path.join(landMaskedS2,'summary_cloud_percent.xlsx')
df.to_excel(outputdir, index=False)  